In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

# Check if SparkSession is created successfully
print(spark)


24/06/27 00:43:05 WARN Utils: Your hostname, MacBook-Pro-Sunith.local resolves to a loopback address: 127.0.0.1; using 10.10.67.15 instead (on interface en0)
24/06/27 00:43:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/27 00:43:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# import pandas as pd

file_path = '/Users/sunith/dss_project/data/DataCoSupplyChainDataset.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)
# description_path = '/Users/sunith/dss_project/data/DescriptionDataCoSupplyChain.csv'
# df_description = spark.read.csv(description_path, header=True, inferSchema=True)
# df = pd.read_csv(file_path, encoding='latin1')
df.show()

24/06/27 00:43:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+--------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+--------------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sale

In [3]:
# Some house cleaning where the the column names are standardized
for col in df.columns:
    new_col = col.replace(' ', '_').lower()
    df = df.withColumnRenamed(col, new_col)

In [4]:

df = df.drop('product_image', 'customer_password', 'customer_email', 
             'customer_fname', 'customer_lname', 'customer_street', 
             'latitude', 'longitude', 'product_status', 'product_description')    
df.createOrReplaceTempView("my_table")
df.show()

+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+--------------+----------------+-----------+----------------+--------------+----------------+-------------+---------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+--------------------+---------------+-------------+---------------+-------------------+------------+-------------+--------------------------+--------------+
|    type|days_for_shipping_(real)|days_for_shipment_(scheduled)|benefit_per_order|sales_per_customer|  delivery_status|late_delivery_risk|category_id| category_name| customer_city|customer_country|customer_id|customer_segment|customer_state|customer_z

In [5]:
# We can see that there are multiple items in one order and they are in multiple rows
df.filter(df.order_id == 67753).show() 

+-----+------------------------+-----------------------------+-----------------+------------------+---------------+------------------+-----------+--------------------+-------------+----------------+-----------+----------------+--------------+----------------+-------------+---------------+------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----------+----------------+----------------------+--------------+-----------+------------+-------------+---------------+-------------------+--------------------+-------------+--------------------------+-------------+
| type|days_for_shipping_(real)|days_for_shipment_(scheduled)|benefit_per_order|sales_per_customer|delivery_status|late_delivery_risk|category_id|       category_name|customer_city|customer_country|customer_id|customer_segment|customer_state|customer_zipcode

In [6]:
spark.sql("Select order_id From my_table GROUP BY order_id having count(order_id) > 1 limit 5").show()

+--------+
|order_id|
+--------+
|   55155|
|   67753|
|   29993|
|   23571|
|   37111|
+--------+



In [12]:
# Pyspark distinct selection from a dataframe
df.select(df.late_delivery_risk).distinct().show()

+------------------+
|late_delivery_risk|
+------------------+
|                 1|
|                 0|
+------------------+



In [13]:
df.filter(df.order_country == 'EE. UU.').show()

+----+------------------------+-----------------------------+-----------------+------------------+---------------+------------------+-----------+-------------+-------------+----------------+-----------+----------------+--------------+----------------+-------------+---------------+------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----+----------------+----------------------+------------+-----------+------------+-------------+---------------+-------------------+------------+-------------+--------------------------+-------------+
|type|days_for_shipping_(real)|days_for_shipment_(scheduled)|benefit_per_order|sales_per_customer|delivery_status|late_delivery_risk|category_id|category_name|customer_city|customer_country|customer_id|customer_segment|customer_state|customer_zipcode|department_id|department_name|m

In [9]:
df.groupBy('order_status').count().orderBy('count', ascending=False).show()

+---------------+-----+
|   order_status|count|
+---------------+-----+
|       COMPLETE|59491|
|PENDING_PAYMENT|39832|
|     PROCESSING|21902|
|        PENDING|20227|
|         CLOSED|19616|
|        ON_HOLD| 9804|
|SUSPECTED_FRAUD| 4062|
|       CANCELED| 3692|
| PAYMENT_REVIEW| 1893|
+---------------+-----+



In [10]:
df.groupBy('order_country').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|       order_country|count|
+--------------------+-----+
|      Estados Unidos|24840|
|             Francia|13222|
|              M�xico|13172|
|            Alemania| 9564|
|           Australia| 8497|
|              Brasil| 7987|
|         Reino Unido| 7302|
|               China| 5758|
|              Italia| 4989|
|               India| 4783|
|           Indonesia| 4204|
|              Espa�a| 3868|
|         El Salvador| 3726|
|Rep�blica Dominicana| 3669|
|            Honduras| 3629|
|                Cuba| 3534|
|             Turqu�a| 3395|
|           Nicaragua| 3046|
|           Guatemala| 2778|
|             Nigeria| 2309|
+--------------------+-----+
only showing top 20 rows



In [11]:
spark.sql("SELECT order_status, count(order_status) as count From my_table group by order_status").show()

+---------------+-----+
|   order_status|count|
+---------------+-----+
|PENDING_PAYMENT|39832|
|       COMPLETE|59491|
|        ON_HOLD| 9804|
| PAYMENT_REVIEW| 1893|
|     PROCESSING|21902|
|         CLOSED|19616|
|SUSPECTED_FRAUD| 4062|
|        PENDING|20227|
|       CANCELED| 3692|
+---------------+-----+

